<a href="https://colab.research.google.com/github/joshlensing/Project-1-Preprocessing/blob/main/Outlet_Sales_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing Data for Outlet Sales Predictions

- Name: Joshua Lensing


Using the sales predictions dataset, I will prepare the data set to create models in order to make predictions for future outlet sales. I first need to clean the data and get it ready for a train test split. From there, I can transform the data get it ready to run through different models.

In [43]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import set_config
set_config(display='diagram')

In [44]:
filename = '/content/sales_predictions (1).csv'
df_sales = pd.read_csv(filename)
df_sales.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


From here, I want to find some general information about my data in order to better understand what I have to work with.

In [45]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [46]:
df_sales.shape

(8523, 12)

It appears that there is some missing data, but I will further check that out with df_sales.isna.sum().

In [47]:
df_sales.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [48]:
df_sales.isna().sum().sum()

3873

There are 3,873 missing values in the dataset. From the df_sales.info(), I can see that the missing values are in an 'object' column and a 'float' column. This will be good to know later for when I will be imputing values and what methods I will need to use for that. Next I want to check to see if there are any duplcaited rows in the dataset and drop them if they are present.

In [49]:
df_sales.duplicated().sum()

0

There are no duplicated rows in the dataset. Next I will check for inconsistencies of values in the categorical columns by creating function to loop through the columns. But before I do that, I want to drop a couple rows that have all unique values that don't provide necessary information to the dataset. 

In [50]:
df_sales.drop(columns=['Item_Identifier', 'Outlet_Identifier'], inplace=True)
df_sales.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,9.30,Low Fat,0.016047,Dairy,249.8092,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,5.92,Regular,0.019278,Soft Drinks,48.2692,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,17.50,Low Fat,0.016760,Meat,141.6180,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,1998,NaN,Tier 3,Grocery Store,732.3800
4,8.93,Low Fat,0.000000,Household,53.8614,1987,High,Tier 3,Supermarket Type1,994.7052


In [51]:
dtypes = df_sales.dtypes
str_cols = dtypes[dtypes=='object'].index
str_cols

Index(['Item_Fat_Content', 'Item_Type', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type'],
      dtype='object')

In [52]:
for col in str_cols:
  print(f'- Column = {col}')
  print(df_sales[col].value_counts(dropna=False))
  print('\n')

- Column = Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64


- Column = Item_Type
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64


- Column = Outlet_Size
Medium    2793
NaN       2410
Small     2388
High       932
Name: Outlet_Size, dtype: int64


- Column = Outlet_Location_Type
Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64


- Column = Outlet_Type
Supermarket Type1    5577
Grocery Store        1083
Supermar

'Item_Fat_Content' is the only categorical column with inconsistent values. I will change the names with a dictionary to make them consistent.

In [53]:
df_sales.replace(['LF', 'low fat'], ['Low Fat', 'Low Fat'], inplace=True)
df_sales.replace(['reg'], ['Regular'], inplace=True)
df_sales['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

I'm almost ready to split the data into a training set and a test set. I seem to have 2 ordinal columns, so I will first need to ordinal encode them first before doing so.

In [54]:
df_sales['Outlet_Size'].replace({'Small': 0, 'Medium': 1, 'High': 2}, inplace=True)
df_sales['Outlet_Location_Type'].replace({'Tier 1': 0, 'Tier 2': 1, 'Tier 3': 2}, inplace=True)
size = df_sales['Outlet_Size'].value_counts()
type = df_sales['Outlet_Location_Type'].value_counts()
print(size, '\n')
print(type)

1.0    2793
0.0    2388
2.0     932
Name: Outlet_Size, dtype: int64 

2    3350
1    2785
0    2388
Name: Outlet_Location_Type, dtype: int64


I am now ready to split the data into training and testing data.

In [55]:
X = df_sales.drop(columns=['Item_Outlet_Sales'])
y = df_sales['Item_Outlet_Sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [56]:
X_train.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
4776,16.350,Low Fat,0.029565,Household,256.4646,2009,1.0,2,Supermarket Type2
7510,15.250,Regular,0.000000,Snack Foods,179.7660,2009,1.0,2,Supermarket Type2
5828,12.350,Regular,0.158716,Meat,157.2946,1999,1.0,0,Supermarket Type1
5327,7.975,Low Fat,0.014628,Baking Goods,82.3250,2004,0.0,1,Supermarket Type1
4810,19.350,Low Fat,0.016645,Frozen Foods,120.9098,2002,NaN,1,Supermarket Type1


In [57]:
# I will instantiate my column selectors
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

After I have selected columns of both object and numeric types, next I will use SimpleImputer to fill in the missing values. Directly after that, I will scale the numeric values and one hot encode the categorical data.

In [58]:
# Imputers (for both numeric and categorical columns)
freq_imputer = SimpleImputer(strategy='most_frequent')
mean_imputer = SimpleImputer(strategy='mean')
# Scaler for numeric
scaler = StandardScaler()
# One-hot encoder for categorical
ohe = OneHotEncoder(handle_unknown='ignore')

I chose the 'most_frequent' for the categorical columns and the 'mean' for the numeric columns. The most frequent value makes the most sense since there are many rows of data and the likelihood of it being that frequent value is higher. The mean makes the most sense for the numeric values because the data contains a lot of specific values, such as prices and weights. Numbers that contain decimal places and that contain high variability work best if imputed as an average.

Next I will create the pipelines for both of the numeric and categorical columns.

In [61]:
# numeric pipeline
numeric_pipe = make_pipeline(mean_imputer, scaler)
numeric_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [62]:
# categorical pipeline
categorical_pipe = make_pipeline(freq_imputer, ohe)
categorical_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder', OneHotEncoder(handle_unknown='ignore'))])

In [63]:
# Tuples for Column Transformer
number_tuple = (numeric_pipe, num_selector)
category_tuple = (categorical_pipe, cat_selector)

Using the tuples created for both the numeric and categorical columns, I will transform the data in ColumnTransformer

In [64]:
# ColumnTransformer
preprocessor = make_column_transformer(number_tuple, category_tuple)
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fe0a4f18bb0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fe0a4f18100>)])

Now all that's left to do is wrap it up and fit it on the training data!

In [65]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fe0a4f18bb0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fe0a4f18100>)])

Now it can be applied to the training and test data in order to transform them.

In [66]:
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

I can check for any missing values now that the columns have been transformed. Then I can run the transformed training set to view the data. The resulting dataset will appear as a numpy array.

In [67]:
np.isnan(X_train_processed).sum().sum()

0

There are no missing values in the training data.

In [68]:
X_train_processed[0:10]

array([[ 8.17248678e-01, -7.12775072e-01,  1.82810922e+00,
         1.32784893e+00,  4.07785364e-01,  1.08494779e+00,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00],
       [ 5.56339503e-01, -1.29105225e+00,  6.03368881e-01,
         1.32784893e+00,  4.07785364e-01,  1.08494779e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00

The data appears to be scaled as well. This dataset has now been preprocessed successfully and is ready for modeling.